Time Calibration of the Mu2e Calorimeter

by Giacinto boccia

version 0.1 | 2024-09-05

In [42]:
import numpy as np
import awkward as ak
import uproot
import quantities as pq
from concurrent.futures import ProcessPoolExecutor

In [43]:
#Input
hits_path = input("Hits file to process:")
cal_path = input("Starting caibration file:") or False
n_runs = input("Iterations to perform:")
#Name of the tree inside the file
hits_path += ":sidet"

In [96]:
#Opening the calibration start-point file
if cal_path:
    print("Starting calibration file not yet supported")
    cal_corection = np.zeros((36, 28, 2)) * pq.ns
else:
    cal_corection = np.zeros((36, 28, 2)) * pq.ns

In [97]:
#Loading the tree, start by defining custom event class that can compute residurals
class Cosmic(ak.Record):     
    def t_residuals(self, correction) -> dict | None:
        if hasattr(self, 'slope'):
            #For each event, returns each hit's residual
            cos_theta = 1 / np.sqrt(1 + self.slope)
            t_arr = np.array(self.Tval + self.templTime) * pq.ns
            t_res = dict()
            for i, row, col, sipm in enumerate(zip(self.iRow, self.iCol, self.SiPM)):
                #Apply current time corrections
                t_arr[i] -= correction[row, col, sipm]
            t_0_ev = np.average(t_arr, weights= self.Qval)
            for time, y, row, col, sipm in zip(t_arr, self.Yval, self.iRow, self.iCol, self.SiPM):
                #Each residual is stored in the dictionary with (row, col, sipm) as key
                t_res[(row, col, sipm)] = time + y / (pq.c * cos_theta) - t_0_ev
            return t_res
        else:
            return None                
ak.behavior["cosmic"] = Cosmic
        
#Loading tree in an array structure, we only need some of the branches
branches = ("nrun", "nsubrun", "evnum", "nHits", "iRow", "iCol", "SiPM", "Xval", "Yval", "Qval", "Tval", "templTime")
with uproot.open(hits_path) as file:
    tree = file.arrays(filter_name = branches, entry_stop= 10, )
    
#Now change the array so that it uses the custom class defned above
tree = ak.Array(tree, with_name= "cosmic")

In [74]:
#To get the parameters, each event is fitted
def linear_fit(event) -> dict[str, np.double | int]:
    x_arr = event.Xval
    y_arr = event.Yval
    if event.nHits > 1:
        #Events that are not empty
        if np.max(x_arr) - np.min(x_arr) > 34.4:
            #Events that are not vertical
            [slope, intercept], residuals, _, _, _ = np.polyfit(x_arr, y_arr, deg= 1, full= True)
            chi_sq : np.double = np.sum(residuals)
            ndf = event.nHits - 2
            return {'vertical' : False, 'slope' : slope, 'intercept' : intercept, 'chi_sq' : chi_sq, 'ndf' : ndf}
        else:
            #Vertical events get flagged
            return {'Vertical': True}
    else:
        return None
        
with ProcessPoolExecutor() as executor:
    fit_results = list(executor.map(linear_fit, tree))
    
fit_results = ak.Array(fit_results)

In [77]:
#Add the fit results to the tree
tree = ak.with_field(tree, fit_results.vertical, "vertical")
tree = ak.with_field(tree, fit_results.slope, "slope")
tree = ak.with_field(tree, fit_results.intercept, "intercept")
tree = ak.with_field(tree, fit_results.chi_sq, "chi_sq")
tree = ak.with_field(tree, fit_results.ndf, "ndf")

In [ ]:
#Apply the filters and compute the residuals